In [12]:
ma_list = [8, 12]

In [1]:
import plotly.graph_objects as go
import pandas as pd

In [3]:
df = pd.read_csv('BTC_USD_1m_Nov.csv')

In [4]:
print(df.head())

                   open_time       open       high        low      close  \
0  2025-11-01 00:00:00+00:00  109608.01  109621.62  109608.00  109616.35   
1  2025-11-01 00:01:00+00:00  109616.34  109639.64  109550.46  109550.47   
2  2025-11-01 00:02:00+00:00  109550.46  109550.46  109470.92  109470.93   
3  2025-11-01 00:03:00+00:00  109470.92  109476.12  109447.00  109447.00   
4  2025-11-01 00:04:00+00:00  109447.00  109457.94  109441.26  109444.44   

     volume                        close_time  quote_volume  n_trades  \
0   5.65926  2025-11-01 00:00:59.999999+00:00  6.203273e+05      1005   
1  47.19536  2025-11-01 00:01:59.999999+00:00  5.172298e+06      2016   
2  10.29554  2025-11-01 00:02:59.999999+00:00  1.127423e+06      2208   
3  39.15662  2025-11-01 00:03:59.999999+00:00  4.286188e+06      2785   
4   4.47815  2025-11-01 00:04:59.999999+00:00  4.901268e+05      1443   

   taker_buy_base  taker_buy_quote  
0         3.81740    418426.015601  
1         1.86014    203897.08

In [5]:
df_plot = df.iloc[0:250].copy()

In [6]:
df_MA = df_plot[['open_time','close']].copy()

In [13]:
for ma in ma_list:
    df_MA[f'MA_{ma}'] = df_MA.close.rolling(window=ma).mean()

In [16]:
df_MA.dropna(inplace=True)

In [30]:
df_MA.head()

,open_time,close,MA_8,MA_10,MA_12,MA_15,DIFF,DIFF_PREV,IS_TRADE
0,2025-11-01 00:25:00+00:00,109519.99,109479.51000,109483.437,109479.613333,109476.360000,-0.103333,NaN,False
1,2025-11-01 00:26:00+00:00,109539.58,109484.18250,109489.103,109485.291667,109481.120667,-1.109167,-0.103333,False
2,2025-11-01 00:27:00+00:00,109531.94,109492.92375,109490.760,109492.157500,109486.555333,0.766250,-1.109167,True
3,2025-11-01 00:28:00+00:00,109517.05,109498.67625,109492.245,109495.001667,109489.595333,3.674583,0.766250,False
4,2025-11-01 00:29:00+00:00,109537.24,109505.24000,109499.768,109496.824167,109493.982000,8.415833,3.674583,False


In [ ]:
df_MA['DIFF'] = df_MA.MA_8 - df_MA.MA_12 #calculate a difference between two moving averages when "creating a cross"
df_MA['DIFF_PREV'] = df_MA.DIFF.shift(1) #gives previous row value

In [23]:
def is_trade(row): #function to identify crosses/trades
    if row.DIFF > 0 and row.DIFF_PREV <= 0:
        return True # Golden Cross
    if row.DIFF < 0 and row.DIFF_PREV >= 0:
        return True # Death Cross
    return False

In [ ]:
#Make new column, apply 
df_MA['IS_TRADE'] = df_MA.apply(is_trade, axis=1) #axis means apply fuction to each row
#create dataframe with made trades
df_trades = df_MA[df_MA.IS_TRADE == True].copy()
#create dataframe with made trades
df_trades = df_MA[df_MA.IS_TRADE == True].copy()
df_MA.reset_index(drop=True, inplace=True)
df_trades.shape
df_trades.head()

In [33]:
fig = go.Figure()

fig.add_trace(go.Candlestick(   #Adding candlestick chart
    x=df_plot['open_time'], #candlesticks
    open=df_plot['open'],
    high=df_plot['high'],
    low=df_plot['low'],
    close=df_plot['close'],
    line=dict(width=1), opacity=1, #coloring
    increasing_line_color='#2EC866', 
    decreasing_line_color='#FF3A4C',
    increasing_fillcolor='#24A06B', 
    decreasing_fillcolor='#CC2E3B',
    )) 

for ma in ma_list:
    col = f'MA_{ma}'
    fig.add_trace(go.Scatter(    #Adding Moving Average line
        x=df_plot.open_time,
        y=df_MA[col],
        line=dict(width=2),
        line_shape='spline', #makes smooth curve
        name=col
        ))

fig.update_layout(width=1100, height=450,
                  margin=dict(l=30, r=30, t=30, b=30),
                  font=dict(color='#e1e1e1'), 
                  paper_bgcolor='#1e1e1e',
                  plot_bgcolor='#1e1e1e')

fig.update_xaxes(
    gridcolor='#1f292f',
    showgrid=True,fixedrange=True,rangeslider=dict(visible=False)
)
 
fig.update_yaxes(
    gridcolor='#1f292f',
    showgrid=True
)
fig.show()